# Introduction : datasets et visualisation

## Importations des librairies courantes

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Exemple : Maisons à Boston

## Lecture du fichier

In [ ]:
# Sous windows, le fichier peut être dans le répertoire "Documents"
bos=pd.read_csv("../input/boston.csv")
# On peut aussi iniquer le chemin d'accès, avec le séparateur '/' (même sous Windows)
# par exemple t=pd.read_csv("C:/datas/boston.csv")
# le séparateur par défaut est la virgule

In [ ]:
# Alternative : utilisation du dataset préfini dans sklearn

from sklearn.datasets import load_boston

boston = load_boston()
# On transforme le tableau en dataframe
bos = pd.DataFrame(boston.data, columns=boston.feature_names)
# On ajoute la colonne 'MV', qui est un tableau séparé dans le dataset sklearn (target)
bos['MV'] = boston.target

## Dataframe

Le fichier est lu sous la forme d'un **Dataframe**

Un Dataframe est une table dont les colonnes sont nommées. Les colonnes d'un Dataframe sont des séries (des vecteurs pour simplifier)

In [ ]:
# Visualisation du début du dataframe
bos.head()

In [ ]:
# Fin du Dataframe
bos.tail()

### Interprétation des colonnes

In [ ]:
# Une colonne de la table
bos['MV']

In [ ]:
# Ou une notation équivalente sous forme pointée (si les colonnes sont des identifiants simples)
bos.MV

In [ ]:
# On peut appliquer les opérations usuelles sur les séries

# lignes entre 50 et 55
print(bos.MV[50:55])

# Valeurs min, max, moyenne, ecart type, quantiles
print(bos.MV.min())
print(bos.MV.max())
print(bos.MV.mean())
print(bos.MV.std())
print(np.percentile(bos.MV,[0,25,50,75,100]))

## Opération sur le Dataframe

In [ ]:
# Dimensions de la table
bos.shape

In [ ]:
# Informations sur les types des données
bos.info()

In [ ]:
# Statistiques sur la table
bos.describe()

In [ ]:
# Trier la table par valeur croissante et nombre de chambres
bos.sort_values(['MV','RM'])

In [ ]:
# On peut sélectionner par exemple les maisons près de la rivière et avec plus de 4 pièces
bos[(bos.CHAS == 1) & (bos.RM > 4)]

In [ ]:
# On peut compter le nombre de valeurs, par exemple les maisons près de la rivière ou pas
bos.CHAS.value_counts()

In [ ]:
# Suppression de colonnes, par exemple la colonne CHAS qui ne contient que des valeurs 0 ou 1
# axis=1 pour supprimer des colonnes, axis=0 pour des lignes
bos = bos.drop(['CHAS'], axis=1)

In [ ]:
bos.head()

  
** Pour plus de détails sur Pandas et la manipulation de Dataframes :  
https://github.com/jhroy/tuto-pandas/blob/master/tutoriel.ipynb  
https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python  **
  

## Graphiques simples

On veut afficher le nuage des points (nombre de pieces, valeur)

In [ ]:
plt.figure(figsize=(10,10))                 # Dimensions de la figure
plt.plot(bos.RM, bos.MV,'ro')               # l'option'bo' affiche des ronds bleus
                                            # on peut utiliser 'r.' pour des points rouges, etc ...
# on peut aussi utiliser les graphiques de type "scatter" (similaire) :
# plt.scatter(bos.RM, bos.MV, color='blue')
plt.xlabel('Nb de pieces')                  # Etiquette pour l'axe Ox
plt.ylabel('Valeur')                        # Etiquette pour l'axe Oy
plt.title('Valeur par nb de pieces')        # Titre du graphique

On observe une correlation entre les deux paramètres, ce qui semble logique.
Par ailleurs, on voit plusieurs points correspondant à la valeur maximum 50, qui semblent mal placés ...
On va le vérifier sur un histogramme

In [ ]:
plt.figure(figsize=(12,6))                 
plt.hist(bos.MV, color='blue', bins=100)      # les "bins" représentent le nombre de barres de l'histogramme                
plt.ylabel('Valeur')                        
plt.title('Histogramme des valeurs')      
plt.show

On constate en effet un pic inhabituel pour la valeur 50.
On sélectionne les lignes dont la valeur est 50 :

In [ ]:
bos[bos.MV == 50]

On a 16 lignes :

In [ ]:
bos[bos.MV == 50].count()

Il semble raisonnable d'éliminer ces lignes afin de ne pas pertuber l'analyse avec des valeurs manifestement erronées :

In [ ]:
bos = bos[bos.MV != 50]         # On conserve les lignes pour lesquelles MV est différent de 50

In [ ]:
plt.figure(figsize=(12,6))                 
plt.hist(bos.MV, color='blue', bins=100)      
plt.xlabel('Nb de pieces')                  
plt.ylabel('Valeur')                        
plt.title('Valeur par nb de pieces')      
plt.show

  
** Il existe une multitude d'options, de types de graphiques et de mises en forme.  
Pour plus de détails sur les graphiques et le module matplotlib :  
http://www.python-simple.com/python-matplotlib/matplotlib-intro.php **
  

## Recherche de corrélations

On a vu assez clairement la corrélation entre le nombre de pièces et la valeur.  
On va utiliser la fonction *corr* pour calculer systématiquement le degré de corrélation entre deux paramètres :

In [ ]:
tabcorr = bos.corr()     # on peut utiliser aussi bos.corr(method='pearson') par exemple
print(tabcorr)

On s'intéresse plus précisément à la valeur des maisons :

In [ ]:
correlations = tabcorr.MV
print(correlations)

On élimine la ligne MV elle-même (qui est forcément à 1) :

In [ ]:
correlations = correlations.drop(['MV'],axis=0)
print(correlations)

Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues :

In [ ]:
correlations = abs(correlations)
print(correlations)

In [ ]:
correlations = correlations.sort_values(ascending=False)     # Tri par ordre décroissant
print(correlations)

On voit que la plus forte corrélation concerne le pourcentage de population pauvre du quartier  
Par contre, le taux de criminalité ou la distance aux principaux centres d'emploi sont assez peu corrélés

## Graphiques avancés

**Seaborn** est une librairie de visualisation statistique avancée

In [ ]:
import seaborn as sns

Visualisation des corrélations avec une "carte de températures"

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(bos.corr())

On peut aussi regrouper les paramètres par *clusters* classés par proximité :

In [ ]:
sns.clustermap(bos.corr())

Seaborn peut afficher tous les nuages de points par paire de paramètres :

In [ ]:
sns.pairplot(bos)

On peut tracer une estimation de la densité de probabilité d'une variable :

In [ ]:
sns.distplot(bos.MV)

*Pour plus de détails sur l'estimation de densité avec des noyaux de convolution, on pourra consulter par exemple :  
https://www.researchgate.net/publication/257199361_Convolution_power_kernels_for_density_estimation*

Une autre forme de graphique combine l'affichage d'un nuage de points (du type *scatter*) et des histogrammes des paramètres :

In [ ]:
sns.jointplot(bos.LSTAT,bos.MV)

On peut regrouper les points voisins sous forme d'hexagones, avec une échelle de couleurs fonction de la densité :

In [ ]:
sns.jointplot(bos.LSTAT, bos.MV, kind='hex')

Pour des valeurs continues, la convolution avec un noyau permet de visualiser une estimation des densités :

In [ ]:
sns.jointplot(bos.LSTAT, bos.MV, kind='kde')

** Pour plus de détails, la document seaborn est très intéressante :  
http://seaborn.pydata.org/ **

# Exercice : Prix des maisons du Comté de King (USA)

** Charger le dataset "kc_house_data.csv" et appliquer une démarche similaire à la précédente **

# Exercice : Le vieux Fidèle

Le Old Faithful (« vieux fidèle » en anglais) est un geyser situé dans le parc national de Yellowstone, aux États-Unis. Ce geyser produit l'un des plus grands jets d'eau chaude et de vapeur au monde, avec le Strokkur islandais.
https://www.youtube.com/watch?v=wE8NDuzt8eg
https://www.youtube.com/watch?v=X4zA_YPCyHs

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/OldFaithful1948.jpg/1200px-OldFaithful1948.jpg">

Dans le fichier "faithful.csv", le séparateur est un point-virgule : il faut le préciser

geyser = pd.read_csv('faithful.csv', sep=';')

** Que pouvez-vous observer sur la représentation en nuage de points (de type jointplot) ? **

# Exercice : dites-le avec des fleurs

Dataset sur les espèces d'iris - prédéfini dans seaborn par exemple

In [ ]:
#iris = sns.load_dataset("iris")